In [1]:
import xmltodict
import copy
from collections import OrderedDict

In [102]:
## goal: merge the nio dictionary and the ado dictionary together
## for each concept in the ado dictioanry, 
## 1) if the concept is not in the nio dictioanry, then add the concept to the merged dictionary;
## 2) if the concept id is in the nio dictionary, then go through the variants,
##    if the variant in the ado is not in the nio, then add the variant to the merged dictionary.

In [103]:
# divide the new merged dictionary into three parts -- 
#      create a new blank dictionary and then paste three parts into it.
# 1) the token ids that are mutual in nio and ado, iterate and merge -- the results show that for the mutual token ids
# .   the variants are complete in both dictionaries.
# 2) the token ids that are in the ado but not in the nio, add directly
# 3) the token ids that are in the nio but not in the ado, add direclty 


In [2]:
xml_path = "/Users/yidesdo21/Projects/inputs/dictionary/"

with open(xml_path+"ado_case.xml") as f:
    ado_xml = f.read()
    
with open(xml_path+"nio_case.xml") as f:
    nio_xml = f.read()

ado_parsed = xmltodict.parse(ado_xml)
nio_parsed = xmltodict.parse(nio_xml)
ado_dict = ado_parsed["synonym"]["token"]
nio_dict = nio_parsed["synonym"]["token"]
# merged_dict = copy.deepcopy(nio_dict)

In [3]:
len(nio_dict)

3755

In [4]:
# the NDDUO and bfo - span, snap is tricky, let's focus on the AlzheimerOntology here 
# update: use all the ontological names in the ado.xml
nio_ids = set()
for nio_concept in nio_dict:
#     nio_id = nio_concept["@id"]
    nio_ids.add(nio_concept["@id"])
#     if nio_id.split(":")[0] == "AlzheimerOntology":   
#         nio_ids.add(nio_id)
print(len(nio_ids))    

3755


In [50]:
# # the intersected concepts for ado and nio
# mutual_ids = set()

# for ado_concept in ado_dict:
#     ado_id = ado_concept["@id"]
#     if ado_id.split(":")[0] == "AlzheimerOntology": 
#         if ado_id in nio_ids:
#             mutual_ids.add(ado_id)

In [51]:
# # extract the sub-concepts in the nio that have the mutual ids 
# # so that when dealing with the mutual concepts, we don't need to iterate over all the nio dictionary 
# nio_mutual_concepts = list()
# ado_mutual_concepts = list()

# for nio_concept in nio_dict:
#     nio_id = nio_concept["@id"]
#     if nio_id in mutual_ids:
#         nio_mutual_concepts.append(nio_concept)
        
# for ado_concept in ado_dict:
#     ado_id = ado_concept["@id"]
#     if ado_id in mutual_ids:
#         ado_mutual_concepts.append(ado_concept)

In [52]:
# mutual_concepts = list()
# for nio_mutual_concept in nio_mutual_concepts:
#     mutual_id, mutual_cano = nio_mutual_concept["@id"], nio_mutual_concept["@canonical"]
#     for ado_mutual_concept in ado_mutual_concepts:
#         ado_mutual_id = ado_mutual_concept["@id"]
#         if ado_mutual_id == mutual_id:
# #             print(ado_mutual_id)
#             nio_vars, ado_vars = nio_mutual_concept["variant"], ado_mutual_concept["variant"]
# #             print(nio_vars)
# #             print(ado_vars)
# #             print("----------")
#             mutual_vars = copy.deepcopy(nio_vars)
#             for ado_var in ado_vars:
#                 if ado_var not in nio_vars:
#                     print(ado_var)
#                     mutual_vars.append(ado_var)
    
#     mutual_concept = OrderedDict([('@id', mutual_id),
#                                   ('@canonical', mutual_cano),
#                                   ('variant', mutual_vars)])    
#     mutual_concepts.append(mutual_concept)

In [5]:
## for all the token ids that are the same in the ado and nio (AlzheimerOntology) -- ignore this 
## all the canononicals are the same if the token ids are the same  -- ignore this 

merged_dict = copy.deepcopy(nio_dict)
corner_case = None
# nio_ids.remove("NDDUO:prevalence")  # avoid duplications in the merged dictionary 

# cnt = 0
for ado_concept in ado_dict:
    ado_id = ado_concept["@id"]
    
    if ado_id not in nio_ids and ado_id != "NDDUO:Prevalence":
        merged_dict.append(ado_concept)
        
    if ado_id == "NDDUO:Prevalence":
        corner_case = ado_concept

for idx, concept in enumerate(merged_dict):
    if concept["@id"] == "NDDUO:prevalence":
        merged_dict[idx] = corner_case    

In [6]:
len(merged_dict)

4847

In [7]:
# turn the dictionary into xml
mydict = {
    'synonym': {
    "token": merged_dict}}

In [8]:
xml_str = xmltodict.unparse(mydict, pretty=True)

In [10]:
# save file
dict_path = "/Users/yidesdo21/Projects/inputs/dictionary/"
save_path_file = "nio_ado_case.xml"    

with open(dict_path+save_path_file, "w") as f:
    f.write(xml_str)